In [1]:
import numpy as np
import open3d as o3d
import math


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def getCenterLines(corners):
    # 육면체의 모든 면 정의 (면의 정점 인덱스)
    faces = [
        [0, 1, 3, 2],
        [4, 5, 7, 6],
        [0, 1, 5, 4],
        [2, 3, 7, 6],
        [0, 2, 6, 4],
        [1, 3, 7, 5]
    ]

    # 각 면의 중심점 계산
    face_total_centers = []
    for face in faces:
        face_coords = [corners[idx] for idx in face]
        total_center = np.mean(face_coords, axis=0)
        face_total_centers.append(total_center)

    # print("face_total_centers = ", face_total_centers)

    # 모든 면 쌍 간 중심점 간 거리 계산
    max_distance = 0
    farthest_face_pair = None
    for i in range(len(faces)):
        for j in range(i + 1, len(faces)):
            distance = np.linalg.norm(face_total_centers[i] - face_total_centers[j])
            if distance > max_distance:
                max_distance = distance
                farthest_face_pair = (i, j)

    # # 가장 먼 면의 중심점을 연결하는 직선 생성
    # farthest_total_center_line = [face_total_centers[farthest_face_pair[0]], face_total_centers[farthest_face_pair[1]]]

    # # 결과 출력
    # print("가장 먼 면의 중심점을 연결하는 직선:", farthest_total_center_line)

    # 정육면체의 선 정보 (꼭지점을 연결하는 라인 인덱스)
    lines = list()
    distances = list()
    temp = list()
    for i in range(0, len(face_total_centers), 2):
        l = [i, (i+1) % len(face_total_centers)]
        lines.append(l)
        d = np.linalg.norm(face_total_centers[i] - face_total_centers[(i+1)])
        distances.append(d)
        p = [list(face_total_centers[i]), list(face_total_centers[i+1])]
        temp.append([d, p, l])

    # print("lines = ", lines)
    # print("distances = ", distances)
    # print("temp = ", temp)
    sorted_lines = sorted(temp, key=lambda x: x[0], reverse=True)
    print("sorted_lines = \n", sorted_lines)

    return sorted_lines

In [3]:

def yaw_pitch_roll_to_rotation_matrix(yaw_degrees, pitch_degrees, roll_degrees):

    yaw_radians = np.radians(yaw_degrees)
    pitch_radians = np.radians(pitch_degrees)
    roll_radians = np.radians(roll_degrees)

    R_x = np.array([[1, 0, 0],
                    [0, np.cos(roll_radians), -np.sin(roll_radians)],
                    [0, np.sin(roll_radians), np.cos(roll_radians)]])

    R_y = np.array([[np.cos(pitch_radians), 0, np.sin(pitch_radians)],
                    [0, 1, 0],
                    [-np.sin(pitch_radians), 0, np.cos(pitch_radians)]])

    R_z = np.array([[np.cos(yaw_radians), -np.sin(yaw_radians), 0],
                    [np.sin(yaw_radians), np.cos(yaw_radians), 0],
                    [0, 0, 1]])

    rotation_matrix = np.dot(np.dot(R_z, R_y), R_x)

    # print(rotation_matrix)

    return rotation_matrix

# 예시 각도 (단위: 도)
# yaw_angle = 30
# pitch_angle = 45
# roll_angle = 60
yaw_angle = 10
pitch_angle = 20
roll_angle = 30

# 회전 행렬 생성
rotation_matrix2 = yaw_pitch_roll_to_rotation_matrix(yaw_angle, pitch_angle, roll_angle)
print(rotation_matrix2)

[[ 0.92541658  0.01802831  0.37852231]
 [ 0.16317591  0.88256412 -0.44096961]
 [-0.34202014  0.46984631  0.81379768]]


In [4]:

def rotation_matrix_to_yaw_pitch_roll(rotation_matrix):
    # Yaw 각도 계산
    yaw = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])

    # Pitch 각도 계산
    pitch = np.arctan2(-rotation_matrix[2, 0], np.sqrt(rotation_matrix[2, 1]**2 + rotation_matrix[2, 2]**2))

    # Roll 각도 계산
    roll = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])

    # 각도를 라디안에서 도로 변환
    yaw = np.degrees(yaw)
    pitch = np.degrees(pitch)
    roll = np.degrees(roll)

    return yaw, pitch, roll

In [5]:
def sortCorners(corners):
    # 각 좌표를 NumPy 배열로 변환
    # coordinates = [np.array(coord) for coord in corners]
    coordinates = corners.copy()
    # print("coordinates = ", coordinates)
    
    # point_0를 임의의 좌표로 설정
    point_0 = coordinates[0]
    # print("- point_0 = ", point_0)

    idx = np.where((coordinates == point_0).all(axis=1))[0][0]
    # print("idx = ", idx)
    coordinates = np.delete(coordinates, idx, axis=0)
    # print("coordinates = ", coordinates)

    # point_0에서 가장 가까운 점을 찾아 point_4로 설정
    point_4 = min(coordinates, key=lambda coord: np.linalg.norm(coord - point_0))
    # print("- point_4 = ", point_4)
    
    idx = np.where((coordinates == point_4).all(axis=1))[0][0]
    # print("idx = ", idx)
    coordinates = np.delete(coordinates, idx, axis=0)
    # print("coordinates = ", coordinates)

    # point_4에서 가장 먼 점을 찾아 point_3으로 설정
    point_3 = max(coordinates, key=lambda coord: np.linalg.norm(coord - point_4))
    # print("- point_3 = ", point_3)
    
    idx = np.where((coordinates == point_3).all(axis=1))[0][0]
    # print("idx = ", idx)
    coordinates = np.delete(coordinates, idx, axis=0)
    # print("coordinates = ", coordinates)

    # point_3에서 가장 가까운 점을 찾아 point_7로 설정
    point_7 = min(coordinates, key=lambda coord: np.linalg.norm(coord - point_3))
    # print("- point_7 = ", point_7)
    
    idx = np.where((coordinates == point_7).all(axis=1))[0][0]
    # print("idx = ", idx)
    coordinates = np.delete(coordinates, idx, axis=0)
    # print("coordinates = ", coordinates)

    # # point_7에서 2번째로 가까운 점을 찾아 point_5로 설정
    # distances_from_point_7 = [np.linalg.norm(coord - point_7) for coord in coordinates]
    # sorted_indices_by_distance = np.argsort(distances_from_point_7)
    # point_5 = coordinates[sorted_indices_by_distance[1]]
    # print("- point_5 = ", point_5)
    
    # point_7에서 가장 가까운 점을 찾아 point_5로 설정 (point_3은 리스트에서 삭제된 상태)
    point_5 = min(coordinates, key=lambda coord: np.linalg.norm(coord - point_7))
    # print("- point_7 = ", point_7)

    idx = np.where((coordinates == point_5).all(axis=1))[0][0]
    # print("idx = ", idx)
    coordinates = np.delete(coordinates, idx, axis=0)
    # print("coordinates = ", coordinates)

    # point_5에서 가장 가까운 점을 찾아 point_1로 설정
    point_1 = min(coordinates, key=lambda coord: np.linalg.norm(coord - point_5))
    # print("- point_1 = ", point_1)
    
    idx = np.where((coordinates == point_1).all(axis=1))[0][0]
    # print("idx = ", idx)
    coordinates = np.delete(coordinates, idx, axis=0)
    # print("coordinates = ", coordinates)

    # point_1에서 가장 먼 점을 찾아 point_6으로 설정
    point_6 = max(coordinates, key=lambda coord: np.linalg.norm(coord - point_1))
    # print("- point_6 = ", point_6)
    
    idx = np.where((coordinates == point_6).all(axis=1))[0][0]
    # print("idx = ", idx)
    coordinates = np.delete(coordinates, idx, axis=0)
    # print("coordinates = ", coordinates)

    # point_6에서 가장 가까운 점을 찾아 point_2로 설정
    # point_2 = min(coordinates, key=lambda coord: np.linalg.norm(coord - point_6))
    point_2 = coordinates[0]
    # print("- point_2 = ", point_2)

    sorted_corners = []
    sorted_corners.append(list(point_0))
    sorted_corners.append(list(point_1))
    sorted_corners.append(list(point_2))
    sorted_corners.append(list(point_3))
    sorted_corners.append(list(point_4))
    sorted_corners.append(list(point_5))
    sorted_corners.append(list(point_6))
    sorted_corners.append(list(point_7))

    # print("sorted_corners = \n", sorted_corners)

    return sorted_corners


In [6]:
# 바운딩 박스의 중심과 모서리 좌표를 사용하여 회전 행렬 계산
# 이 예제에서는 회전을 계산하지만, 실제로 회전 행렬을 추출하는 방법은 문제에 따라 다를 수 있습니다.
# 여기서는 회전을 추정하기 위해 간단하게 회전 행렬을 계산하는 코드를 제공합니다.
def compute_rotation_matrix(corners, total_center):
    cov_matrix = np.cov((corners - total_center).T)
    eigvals, eigvecs = np.linalg.eig(cov_matrix)
    rotation_matrix = eigvecs
    return rotation_matrix


In [9]:

# read ply file
pcd = o3d.io.read_point_cloud('ply_image/17.ply')

################################
#### 바닥 제거

total_aabb = pcd.get_axis_aligned_bounding_box()
total_aabb.color = (1, 0, 0)
total_obb = pcd.get_oriented_bounding_box()
total_obb.color = (0, 1, 0)

# visualize
# o3d.visualization.draw_geometries([pcd, total_aabb, total_obb])

print(total_aabb)
print(total_aabb.get_min_bound())
print(total_aabb.get_max_bound())

total_center = total_aabb.get_max_bound() + total_aabb.get_min_bound()
total_size = total_aabb.get_max_bound() - total_aabb.get_min_bound()
print(total_center)
print(total_size)

# 회전 행렬 참고자료 : https://en.wikipedia.org/wiki/Rotation_matrix
yaw_angle = 0  # + : crop 영역 반시계 방향 회전(카메라 시점), - : crop 영역 시계 방향 회전(카메라 시점)
pitch_angle = 0
roll_angle = 0

rotation_matrix = yaw_pitch_roll_to_rotation_matrix(yaw_angle, pitch_angle, roll_angle)
# total_size = [0.8,  # 너비(카메라 기준, 2d width) 1.139 보다 작아지면 우측 잘리기 시작함, 0.01 = 2 pixel 거리
#         0.6,  # 높이(카메라 기준, 2d height) 0.849 보다 작아지면 아래쪽 잘리기 시작함, 0.00875 = 2 pixel 거리
#         1.387]  # 깊이(카메라 기준 거리, 3d depth) 1.387 보다 작아지면 좌측 아래쪽 바닥 잘리기 시작함
#                 # 바닥 제거 1.345

# total_center = [0.0,  # + : crop 영역이 우측으로 이동, - : crop 영역이 좌측으로 이동 (-0.5195 ~ 0.5708), 합 1.0903
#           0.0,  # + : crop 영역이 위쪽으로 이동, - : crop 영역이 아래쪽으로 이동 ( -0.4377 ~ 0.4179)
#           0.0,] # + : crop 영역이 카메라쪽으로 이동, - : crop 영역이 아래쪽으로 이동

crop_size = [total_size[0], total_size[1], 1.345]
crop_center = [total_center[0], total_center[1], 0]

total_obb = o3d.geometry.OrientedBoundingBox(crop_center, rotation_matrix, crop_size) # or you can use axis aligned bounding box class

print(rotation_matrix)
print(total_size)
print(total_center)

floor_removed_pcd = pcd.crop(total_obb)

###################################
#### 객체 영역 crop
corners = [
    [ 0.00, -0.14,  0.0],   # LB
    [ -0.05, -0.06,  0.0],    # LT
    [ 0.07, 0.03,  0.0],     # RT
    [ 0.14, -0.05,  0.0]     # RB
]
corners = np.array(corners)

# Convert the corners array to have type float64
bounding_polygon = corners.astype("float64")

# Create a SelectionPolygonVolume
vol = o3d.visualization.SelectionPolygonVolume()

# You need to specify what axis to orient the polygon to.
# I choose the "Z" axis. I made the max value the maximum Z of
# the polygon vertices and the min value the minimum Z of the
# polygon vertices.
axis_num = 2
vol.orthogonal_axis = "Z"
# vol.axis_max = np.max(bounding_polygon[:, axis_num])
# vol.axis_min = np.min(bounding_polygon[:, axis_num])
vol.axis_max = 10.0
vol.axis_min = -10.0

# # Set all the Z values to 0 (they aren't needed since we specified what they
# # should be using just vol.axis_max and vol.axis_min).
bounding_polygon[:, axis_num] = 0

# Convert the np.array to a Vector3dVector
vol.bounding_polygon = o3d.utility.Vector3dVector(bounding_polygon)

print("vol = ", vol)
print("vol.orthogonal_axis = ", vol.orthogonal_axis)
print("vol.axis_min = ", vol.axis_min)
print("vol.axis_max = ", vol.axis_max)
print("vol.bounding_polygon = \n", np.asarray(vol.bounding_polygon))


# Crop the point cloud using the Vector3dVector
crop_pcd = vol.crop_point_cloud(floor_removed_pcd)

# 정육면체의 선 정보 (꼭지점을 연결하는 라인 인덱스)
lines = list()
for i in range(len(corners)):
    lines.append([i, (i+1) % len(corners)])

print(lines)

# 라인 정보 생성
lineset = o3d.geometry.LineSet()
lineset.points = o3d.utility.Vector3dVector(corners)
lineset.lines = o3d.utility.Vector2iVector(lines)

object_obb = crop_pcd.get_oriented_bounding_box()
# object_obb.color = (0, 1, 0)
object_obb.color = (1, 0, 0)

####################################
# 내부 라인 그리기

corners = np.asarray(object_obb.get_box_points())
print("corners = \n", corners)

sorted_corners = sortCorners(corners)
print("sorted_corners = \n", sorted_corners)

sorted_lines_info = getCenterLines(sorted_corners)
print("sorted_lines_info = \n", sorted_lines_info)

face_total_centers = list()
lines = list()

for i in range(len(sorted_lines_info)):
    face_total_centers.append(sorted_lines_info[i][1][0])
    face_total_centers.append(sorted_lines_info[i][1][1])
    lines.append(sorted_lines_info[i][2])

# 라인 정보 생성
lineset = o3d.geometry.LineSet()
# lineset.points = o3d.utility.Vector3dVector([list(farthest_total_center_line[0]), list(farthest_total_center_line[1])])
# lineset.lines = o3d.utility.Vector2iVector([[0, 1]])
lineset.points = o3d.utility.Vector3dVector(face_total_centers)
lineset.lines = o3d.utility.Vector2iVector(lines)


#############################################
# 최종 결과

object_rotation_matrix = compute_rotation_matrix(np.asarray(sorted_corners), np.asarray(total_center))

yaw, pitch, roll = rotation_matrix_to_yaw_pitch_roll(object_rotation_matrix)
# print("-- ", yaw, pitch, roll)

center = object_obb.get_max_bound() + object_obb.get_min_bound()
size = object_obb.get_max_bound() - object_obb.get_min_bound()
print(center)
print(size)

x = center[0]
y = center[1]
z = center[2]
print("x:", x)
print("y:", y)
print("z:", z)
print("yaw:", yaw)
print("pitch:", pitch)
print("roll:", roll)

width = sorted_lines_info[0][0]
height = sorted_lines_info[1][0]
depth = sorted_lines_info[2][0]
print("width:", width)
print("height:", height)
print("depth:", depth)

# visualize
# o3d.visualization.draw_geometries([crop_pcd, lineset, total_aabb, object_obb])
o3d.visualization.draw_geometries([pcd, object_obb, lineset])

AxisAlignedBoundingBox: min: (-0.524414, -0.446289, -0.695312), max: (0.570312, 0.41748, -0.50293)
[-0.524414 -0.446289 -0.695312]
[ 0.570312  0.41748  -0.50293 ]
[ 0.045898 -0.028809 -1.198242]
[1.094726 0.863769 0.192382]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[1.094726 0.863769 0.192382]
[ 0.045898 -0.028809 -1.198242]
vol =  SelectionPolygonVolume, access its members:
orthogonal_axis, bounding_polygon, axis_min, axis_max
vol.orthogonal_axis =  Z
vol.axis_min =  -10.0
vol.axis_max =  10.0
vol.bounding_polygon = 
 [[ 0.   -0.14  0.  ]
 [-0.05 -0.06  0.  ]
 [ 0.07  0.03  0.  ]
 [ 0.14 -0.05  0.  ]]
[[0, 1], [1, 2], [2, 3], [3, 0]]
corners = 
 [[ 0.00220767 -0.12714297 -0.70760325]
 [ 0.12593374 -0.04196766 -0.69825132]
 [-0.04486142 -0.06193299 -0.67879699]
 [ 0.01211959 -0.14866984 -0.64267598]
 [ 0.08877656  0.00171545 -0.60451779]
 [-0.03494951 -0.08345986 -0.61386972]
 [ 0.13584566 -0.06349453 -0.63332405]
 [ 0.07886465  0.02324232 -0.66944506]]
sorted_corners = 
 [[0.00220767213445